<a href="https://colab.research.google.com/github/loftmain/PyTorchTutorial/blob/master/cnn_day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('./MyDrive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn import datasets
import numpy.linalg as lin
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist, fashion_mnist
from keras import models
from keras.utils import to_categorical
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [9]:
train_dir = './MyDrive/My Drive/dataset/clean-dataset/train'
validation_dir = './MyDrive/My Drive/dataset/clean-dataset/validation/'
image_size = 224

In [4]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size,3))

58892288/58889256 [==============================] - 2s 0us/step


In [6]:
model = models.Sequential()

model.add(vgg_conv)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 771       
Total params: 21,138,243
Trainable params: 21,138,243
Non-trainable params: 0
_________________________________________________________________


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_batchsize = 20
val_batchsize = 10

In [10]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')

# Data Generator for Validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)



Found 141 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [12]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=3,
                             epochs=5,
                             validation_data=validation_generator,
                             validation_steps=3,
                             verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
3/3 [==============================] - 31s 10s/step - loss: 1.1650 - accuracy: 0.6667 - val_loss: 43880.6172 - val_accuracy: 0.0000e+00
Epoch 2/5
3/3 [==============================] - 3s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 2885613.2500 - val_accuracy: 0.0000e+00
Epoch 3/5
3/3 [==============================] - 4s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 26495254.0000 - val_accuracy: 0.0000e+00
Epoch 4/5
3/3 [==============================] - 1s 228ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 104958224.0000 - val_accuracy: 0.0000e+00
Epoch 5/5
3/3 [==============================] - 2s 532ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 264893728.0000 - val_accuracy: 0.0000e+00


In [14]:
for layer in vgg_conv.layers:
    layer.trainable = True
    
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False    

    
for layer in vgg_conv.layers:
    print(layer.trainable)
    

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True


In [15]:
vgg_conv.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

이것이 fine tunning이다!

In [ ]:
# 1 : 네트워크 구조만 사용
model = models.Sequential()

model.add(vgg_conv)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.summary()

# 2 : 전체 재학습
for I in vgg_conv.layers:
    I.trainable = false
    # 2개의 네트워크를 분리해서 고속으로 학습

# fine tunning : 네크워크 구조 사용, 일부 필터(상위) 레이어의 가중치는 재사용
for i in vgg_conv.layers[:-4]:
    i.trainalbe = False